In [ ]:

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import plotly.figure_factory as ff
import datetime

# Predictions
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
print("Modules are Imported ")


# LOADING THE DATA

In [ ]:
age_data = pd.read_csv('AgeGroupDetails.csv')
icmrTestLabs = pd.read_csv('ICMRTestingLabs.csv') #Test labs data
covid_data = pd.read_csv('covid_19_india.csv')
world_data = pd.read_csv('covid_19_clean_complete.csv')
hospital_data = pd.read_csv('HospitalBedsIndia.csv')

# DATA OF INDIA (COVID)

In [ ]:
covid_data.head() 

In [ ]:
covid_data.tail(5) #Recent Cases 

In [ ]:
 covid_data.isnull().sum() #checking for null values


In [ ]:
state_cases = covid_data.groupby('State/UnionTerritory')['Confirmed','Deaths','Cured'].max().reset_index()

#Creating new columns 'Active', 'Death rate per 100' and 'Cure rate per 100'
state_cases['Active'] = state_cases['Confirmed'] - (state_cases['Cured'] + state_cases['Deaths'])


In [ ]:
#Sort by maximum Confirmed Cases
state_cases = state_cases.sort_values(by='Confirmed', ascending=False) 
state_cases.style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Cured"])\
                        .background_gradient(cmap='Blues', subset=["Active"])\
                        

In [ ]:
#plot for Total Deaths in India (State wise Anlaysis)
import datetime 
today = datetime.date.today() # today date
yesterday = '02/09/20'
df1= covid_data[covid_data['Date']==yesterday] # Below the plot I have shown the dataframe
fig = px.bar(df1, x='State/UnionTerritory', y='Deaths', height=600)
fig.update_layout(
    title='Till {} Total Deaths in India'.format(yesterday))
fig.show()
df1.head()



In [ ]:
# Plot for Recovered Cases in India (State Wise Analysis)

import datetime 
today = datetime.date.today() # today date
yesterday = '02/09/20'
df1= covid_data[covid_data['Date']==yesterday] # Selecting the yesterday date 
fig = px.bar(df1, x='State/UnionTerritory', y='Cured', height=600)
fig.update_layout(
    title='Till {} Total Recovered Cases in India'.format(yesterday))
fig.show() # plot 
df1.head() # Dataframe df1

# AGE DATA

In [ ]:
age_data.head(15) # It shows which age group is more affected by Covid 

In [ ]:
age_data.info()

In [ ]:
percent = age_data['Percentage'] #percentage of people affected
percent


In [ ]:
# Plot 
plt.figure(figsize=(14,8)) #This Shows age group between 20 -29 is most affected by Covid
sns.barplot(data= age_data,x='AgeGroup',y='TotalCases')
plt.title('Age Group Distribution')
plt.xlabel('Age Group')
plt.ylabel('Total Nuumber of  Cases')


# ICMR TEST LAB DATA


In [ ]:
icmrTestLabs.head() #This will show everything bout Labs i.e Name , Address , State , pincode etc.

In [ ]:
icmrTestLabs['state'].value_counts() #Just to Show which state has more Labs



In [ ]:
icmrTestLabs['type'].unique() # Just to check which type of labs i.e Private , Government or Collection Site

In [ ]:
Labs_type_by_State = icmrTestLabs[['type', 'state']].groupby(['type', 'state']).sum()
display(Labs_type_by_State)

Statewise Analysis of Test Labs

In [ ]:
state=list(icmrTestLabs['state'].value_counts().index) #states
count=list(icmrTestLabs['state'].value_counts()) #number of labs
plt.figure(figsize=(14,8))
sns.barplot(x=count,y=state)
plt.xlabel('Counts')
plt.ylabel('States')
plt.title('ICMR Test labs per States')
plt.tight_layout()

# HOSPITAL DATA

In [ ]:
hospital_data.head(10)

In [ ]:
hospital_data.describe()# Mean , total , percentile etc on all columns

In [ ]:
hospital_data.isnull().sum() #checking Null values 
#  our data has only one column which is missing values i.e  NumSubDistrictHospitals_HMIS 


In [ ]:
hospital_data['NumSubDistrictHospitals_HMIS'] = hospital_data['NumSubDistrictHospitals_HMIS'].fillna(0)
#fill all the missing values with zero (0)

In [ ]:
# Public Beds in Hospital  
fig = px.bar(hospital_data,                                  
             x=hospital_data['State/UT'].iloc[:36], 
             y=hospital_data['NumPublicBeds_HMIS'].iloc[:36], 
             color=hospital_data['NumPublicBeds_HMIS'].iloc[:36])

fig.update_layout(title={
        'text': "Num of Public Beds in Each State",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="State/UT",
        yaxis_title="Number of Public Beds",
        plot_bgcolor='white')

<div style="color:white;
           padding:8px 10px 0 10px;
           display:inline-block;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana">
    <h1 style='color:white;'>5. World Data</h1>
</div>

In [ ]:
world_data.head() #World Data

<div style="color:white;
           padding:8px 10px 0 10px;
           display:inline-block;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana">
    <h1 style='color:white;'>6. Making Predictions (India)</h1>
</div>

 Data forecasting is done by using Prophet library .Prophet is an open source library published by Facebook which is good for Time Series Forecasting. We will predict the coronavirus cases till ** Mid August  2020**.

In [ ]:
india_data = world_data[world_data["Country/Region"]=="India"] #Selecting India from World Data
india_data.head()

In [ ]:
pred_data = india_data.groupby('Date')[['Confirmed', 'Recovered', 'Deaths']].sum().reset_index()
pred_data['Date'] = pred_data['Date'].apply(pd.to_datetime, dayfirst=True)
#Grouping the data Based upon Date and Cnfirmed , Recovered Casea and Deaths

<div style="color:white;
           padding:8px 10px 0 10px;
           display:inline-block;
           border-radius:5px;
           background-color:#EC2566;
           font-size:90%;
           font-family:Verdana">
    <h1 style='color:white;'>6.1. Confirmed Cases in India</h1>
</div>

In [ ]:
pred_confirm = pred_data.loc[:, ['Date', 'Confirmed']] # for Prophet model we are required to have atleast 2 columns
# Date and Object which we are  Predicting
pred_confirm.tail()

<div style="color:white;
           padding:8px 10px 0 10px;
           display:inline-block;
           border-radius:5px;
           background-color:#5E7B81;
           font-size:90%;
           font-family:Verdana">
    <h1 style='color:#ffffff;'>6.1.1. Creating Model</h1>
</div>

Defining our Prophet() model.

In [ ]:
model = Prophet()

In [ ]:
# Dataframe must have columns "ds" and "y" with the dates and values  for prophet prediction
pred_confirm.columns = ['ds', 'y']
model.fit(pred_confirm)

In [ ]:
future = model.make_future_dataframe(periods=45) # helper function to extend the dataframe for specified days
future.tail()

<div style="color:white;
           padding:8px 10px 0 10px;
           display:inline-block;
           border-radius:5px;
           background-color:#5E7B81;
           font-size:90%;
           font-family:Verdana">
    <h1 style='color:#ffffff;'>6.1.2. Making Predictions</h1>
</div>

In [ ]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_conf = model.predict(future)
forecast_india_conf

In [ ]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig) 

In [ ]:
fig = model.plot(forecast_india_conf)

In [ ]:
# Plot the various component i.e how the trend goes 
fig = model.plot_components(forecast_india_conf)

In [ ]:
cnfrm = forecast_india_conf.loc[:,['ds','trend']]
cnfrm = cnfrm[cnfrm['trend']>0] # trend goes like present trend
cnfrm.columns = ['Date','Confirm'] # name the columns
cnfrm.tail(15)

# Recovered Cases


In [ ]:
pred_recovered_cases = pred_data.loc[:, ['Date', 'Recovered']]

pred_recovered_cases.head(10)


In [ ]:
model = Prophet()

pred_recovered_cases.columns = ['ds', 'y']
model.fit(pred_recovered_cases)

In [ ]:
future = model.make_future_dataframe(periods=45)
future.tail()

In [ ]:
forecast_india_recover = model.predict(future)

forecast_india_recover

In [ ]:
fig = plot_plotly(model, forecast_india_recover)
fig.update_layout(template='plotly_white')
iplot(fig) 

In [ ]:
pred_deaths = pred_data.loc[:, ['Date', 'Deaths']]
pred_deaths.tail(3)

In [ ]:
model = Prophet()
pred_deaths.columns = ['ds', 'y']
model.fit(pred_deaths)

In [ ]:
future = model.make_future_dataframe(periods=45)
future.tail()

In [ ]:
forecast_india_death = model.predict(future)
forecast_india_death

In [ ]:
fig = plot_plotly(model, forecast_india_death)
fig.update_layout(template='plotly_white')
iplot(fig) 